In [48]:
# Import Libraries
import pickle

import mlflow
import pandas as pd
import xgboost
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from mlflow.tracking import MlflowClient
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error


In [49]:
# defining a function to quickly read and prepare data
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['hour'] = df.lpep_pickup_datetime.dt.hour
    df['dayofweek'] = df.lpep_pickup_datetime.dt.day_of_week

    categorical = ['PULocationID', 'DOLocationID', 'hour', 'dayofweek', 'VendorID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [50]:
# Reading and preparing the dataset
df_train = read_dataframe('/home/ubuntu/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/ubuntu/data/green_tripdata_2021-02.parquet')

In [51]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment-latest")

<Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='nyc-taxi-experiment-latest', tags={}>

In [52]:
# defing feature types
categorical = ['PU_DO', 'hour', 'dayofweek', 'VendorID'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# applying one hot encoding
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

with open("models/preprocessor/dvtransformer.bin", 'wb') as fileout:
    pickle.dump(dv, fileout)

In [53]:
# defining the target
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [54]:
# defining xboost level data

training_data = xgboost.DMatrix(X_train, label=y_train)
valid = xgboost.DMatrix(X_val, label=y_val)

In [55]:
def objective(params):
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        model = xgboost.train(
            params = params,
            dtrain = training_data,
            num_boost_round = 120,
            evals = [(valid, "validation")],
            early_stopping_rounds = 20
        )

        y_pred = model.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -6, 1),
    'reg_lambda': hp.loguniform('reg_lambda', -5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective' : 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=15,
    trials=Trials()
)



[0]	validation-rmse:15.33084                          
[1]	validation-rmse:11.62856                          
[2]	validation-rmse:9.35894                           
[3]	validation-rmse:8.03920                           
[4]	validation-rmse:7.31255                           
[5]	validation-rmse:6.88714                           
[6]	validation-rmse:6.64969                           
[7]	validation-rmse:6.49991                           
[8]	validation-rmse:6.40741                           
[9]	validation-rmse:6.36009                           
[10]	validation-rmse:6.31988                          
[11]	validation-rmse:6.30188                          
[12]	validation-rmse:6.27914                          
[13]	validation-rmse:6.26231                          
[14]	validation-rmse:6.25479                          
[15]	validation-rmse:6.24417                          
[16]	validation-rmse:6.23809                          
[17]	validation-rmse:6.23394                          
[18]	valid

In [56]:
# Train the best model
best_result['max_depth'] = int(best_result['max_depth'])
best_result['objective'] = 'reg:squarederror'
best_result['seed'] = 42

mlflow.xgboost.autolog(disable=True)

best_result['max_depth'] = int(best_result['max_depth'])
best_result['objective'] = 'reg:squarederror'
best_result['seed'] = 42

mlflow.xgboost.autolog(disable=True)

best_result['max_depth'] = int(best_result['max_depth'])
best_result['objective'] = 'reg:squarederror'
best_result['seed'] = 42

mlflow.xgboost.autolog(disable=True)

best_result['max_depth'] = int(best_result['max_depth'])
best_result['objective'] = 'reg:squarederror'
best_result['seed'] = 42

#mlflow.xgboost.autolog()
with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(best_result)
    model = xgboost.train(
            params = best_result,
            dtrain = training_data,
            num_boost_round = 300,
            evals = [(valid, "validation")],
            early_stopping_rounds = 50
        )
    y_pred = model.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/preprocessor/dvtransformer.bin",
                        artifact_path="preprocessor")
    mlflow.xgboost.log_model(xgb_model=model,artifact_path="model")
            


[0]	validation-rmse:17.73984
[1]	validation-rmse:14.99203
[2]	validation-rmse:12.83501
[3]	validation-rmse:11.15678
[4]	validation-rmse:9.87532
[5]	validation-rmse:8.90616
[6]	validation-rmse:8.18117
[7]	validation-rmse:7.65649
[8]	validation-rmse:7.26646
[9]	validation-rmse:6.98155
[10]	validation-rmse:6.77881
[11]	validation-rmse:6.62508
[12]	validation-rmse:6.51610
[13]	validation-rmse:6.43205
[14]	validation-rmse:6.36548
[15]	validation-rmse:6.31823
[16]	validation-rmse:6.27979
[17]	validation-rmse:6.25160
[18]	validation-rmse:6.22863
[19]	validation-rmse:6.21061
[20]	validation-rmse:6.19560
[21]	validation-rmse:6.18419
[22]	validation-rmse:6.16929
[23]	validation-rmse:6.16177
[24]	validation-rmse:6.15205
[25]	validation-rmse:6.14616
[26]	validation-rmse:6.14205
[27]	validation-rmse:6.13832
[28]	validation-rmse:6.13271
[29]	validation-rmse:6.12942
[30]	validation-rmse:6.12644
[31]	validation-rmse:6.12425
[32]	validation-rmse:6.12158
[33]	validation-rmse:6.12029
[34]	validation-rmse

In [57]:
#Load the model and artifact
run_id = '8578b756f8694873a015fbb0e0ff4a16'
model_uri = f'runs:/{run_id}/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

print(loaded_model)


y_pred = loaded_model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(rmse)

# load dict vectoriser
client = MlflowClient()
client.download_artifacts(run_id, "preprocessor", "models")

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 8578b756f8694873a015fbb0e0ff4a16

6.048011580611901


'/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/models/preprocessor'